In [2]:
import minsearch
import json

In [3]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

# Flatten the documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # SELECT * WHERE course = 'data-engineering-zoomcamp'; #kind of SQL query for keyword_fields
)

In [7]:
q = "the course has already started, can i still enroll?" #question example

In [8]:
index.fit(documents)

In [9]:
# All these fields como from elastic search and buy default they are all equal to 1.0

boost = {
    "question": 3.0, #the question field is 3 times mr important than the other fields
    'section':0.5 # this filed is 0.5 times less important than the other fields
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
    boost_dict=boost,
    num_results=5
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
from openai import OpenAI
import os

In [12]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": "is it too late to join the course?"
         },
    ]
)

In [14]:
response.choices[0].message.content

'It depends on several factors such as the specific course you are interested in, its enrollment deadlines, and the policies of the institution offering the course. Here are a few steps you can take to find out:\n\n1. **Check the Course Website**: Look for any information about enrollment deadlines or the academic calendar.\n   \n2. **Contact the Institution**: Reach out to the admissions office or the specific department offering the course. They can provide you with the most accurate information.\n\n3. **Look for Late Enrollment Options**: Some institutions offer late enrollment or add/drop periods where you can still join a course after the official start date.\n\n4. **Consider Online Courses**: Many online platforms offer flexible enrollment options and might not adhere to a traditional academic calendar.\n\nIf you provide more specific details, I can offer more targeted advice.'

In [15]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: {question}

CONTEXT:{context}

""".strip()

In [16]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [17]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [18]:
prompt = prompt_template.format(question=q, context=context).strip()

In [19]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT:section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [20]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": prompt
         },
    ]
)

In [21]:
print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homework. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


## MORE MODULAR

In [36]:
def search(query):
    boost = {
        "question": 3.0, #the question field is 3 times mr important than the other fields
        'section':0.5 # this filed is 0.5 times less important than the other fields
    }

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
        boost_dict=boost,
        num_results=10
    )
    
    return results

In [23]:
# verify the search function
search("How do i run Kafta?")

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial tha

In [24]:
def build_prompt(query,search_results):
    
    prompt_template = """
    You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
    Use only the facts from CONTEXT when answering when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, out NONE

    QUESTION: {question}

    CONTEXT:{context}

    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [28]:
def llm(prompt):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    ) 
    
    return response.choices[0].message.content

In [37]:
query = "how do i run kafka?"
search_results = search(query)
prompt = build_prompt(query,search_results)
answer = llm(prompt)

In [38]:
print(answer)

To run Kafka, the answer varies depending on if you're working with Java or Python. Here are specific instructions based on the context provided:

### Java Kafka:
In the project directory, run:
```sh
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

### Python Kafka:
1. If you encounter a "Module 'kafka' not found" error, create a virtual environment and run the required Python files in that environment:
   ```sh
   python -m venv env
   source env/bin/activate
   pip install -r ../requirements.txt
   ```
   On Windows, activate with:
   ```sh
   env\Scripts\activate
   ```
   To deactivate the virtual environment, use:
   ```sh
   deactivate
   ```

2. Ensure your Kafka broker container is running. If you encounter the `kafka.errors.NoBrokersAvailable` error, confirm your Kafka broker docker container is active with:
   ```sh
   docker ps
   ```
   In the folder containing your `docker-compose.yml` file, run:
   ```sh
   docker compos

In [42]:
query = "how do i run kafka?"

def rag(query):
    search_results = search(query) # we can replace this other search function (ex: elastic search)
    prompt = build_prompt(query,search_results) #we can optimize the prompt function
    answer = llm(prompt) #we can use another llm model (ex: mistral, ada, etc.)
    
    return answer

In [41]:
rag('The course has already started, can i still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homework. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [ ]:
# we had this custom search function, because in the future we can replace it with another search function (ex: elastic search) and also we sometimes is not easy to use elastic search with other cloud providers (ex: saturn cloud, etc. - sponsor)